# 第10章：決定木

## 10.1 決定木のハイパーパラメータを変更してAUCを向上できないか試してみよう。

### 【解答例】

- 途中までは書籍本文と同様の処理を実行する。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import train_test_split # ホールドアウト用モジュール
from sklearn.tree import DecisionTreeClassifier as dtc # 決定木用モジュール
from sklearn.tree import plot_tree # 決定木の可視化モジュール
from sklearn.metrics import accuracy_score, roc_auc_score # 評価指標用モジュール

In [ ]:
# 前処理済み中間データのdictを読み取る
pp_data_dict = joblib.load("../intermediate/pp_data_dict.pkl3")

# 辞書型変数の値に格納された決定木向け中間データを読み取る
dtc_train_df = pp_data_dict["dtc"]["train"]
dtc_test_df = pp_data_dict["dtc"]["test"]

In [ ]:
# 6:4の割合でホールドアウト法を行う.
dtc_train_train_df, dtc_train_valid_df = train_test_split(dtc_train_df, test_size=0.4, random_state=57, shuffle=True)

In [ ]:
dtc_train_df.shape, dtc_train_train_df.shape, dtc_train_valid_df.shape, dtc_test_df.shape

In [ ]:
# 目的変数をtargetという変数に格納する
target = "buy_flag"

# 説明変数をfeaturesという変数に格納する
dtc_features = dtc_train_df.columns.tolist()

# customer_idとbuy_flagは説明変数ではない為削除する
dtc_features.remove("customer_id")
dtc_features.remove("buy_flag")

- ここから章末問題独自のプログラムになる
- グリッドサーチを行うハイパーパラメータとして、書籍本編では「最大の木の深さ」（max_depth）と「ランダムの種」（random_state=固定値で変化なし）を扱ったが、章末問題では以下の2つを変化させてみよう。
    - 分割に必要な各ノードの最小のサンプルサイズ（min_samples_split）：2～5の範囲
    - 各ノードに必要な最小のサンプルサイズ（min_samples_leaf）：1～5の範囲

In [ ]:
# ハイパーパラメータチューニング
# グリッドサーチ
# 探索するハイパーパラメータの候補をリストに格納する.
dtc_params_list = [{"min_samples_split": i, "min_samples_leaf": j} for i in np.arange(2, 6, 1) for j in np.arange(1, 6, 1)]
dtc_params_list

In [ ]:
# グリッドサーチの結果を格納するリスト
gs_result_list = []

# グリッドサーチの実施
for params_dict in dtc_params_list:
    # 定義する
    tmp_dtc_clf = dtc(**params_dict)
    # 学習する
    tmp_dtc_clf.fit(dtc_train_train_df[dtc_features], dtc_train_train_df[target])
    
    # train-trainを予測する（predict関数）
    train_pred_y = tmp_dtc_clf.predict(dtc_train_train_df[dtc_features])
    
    # train-trainを予測する（predict_proba関数）
    train_proba_y = tmp_dtc_clf.predict_proba(dtc_train_train_df[dtc_features]).T[1]
    
    # train-validを予測する（predict関数）
    valid_pred_y = tmp_dtc_clf.predict(dtc_train_valid_df[dtc_features])
    
    # train-validを予測する（predict_proba関数）
    valid_proba_y = tmp_dtc_clf.predict_proba(dtc_train_valid_df[dtc_features]).T[1]
    
    # train-trainを検証する
    train_accuracy_val = accuracy_score(dtc_train_train_df[target], train_pred_y)
    train_auc_val = roc_auc_score(dtc_train_train_df[target], train_proba_y)
    
    # train-validを検証する
    valid_accuracy_val = accuracy_score(dtc_train_valid_df[target], valid_pred_y)
    valid_auc_val = roc_auc_score(dtc_train_valid_df[target], valid_proba_y)
    
    # リストに格納する
    gs_result_list += [[params_dict, train_accuracy_val, train_auc_val, valid_accuracy_val, valid_auc_val]]

In [ ]:
# 結果をDataFrameに格納する
gs_result_df = pd.DataFrame(gs_result_list, columns=["params_dict", "train_accuracy", "train_auc", "valid_accuracy", "valid_auc"])

# 結果を確認する
gs_result_df[["params_dict", "train_accuracy", "train_auc", "valid_accuracy", "valid_auc"]].sort_values(by="valid_auc", ascending=False).reset_index(drop=True)

- 以上から、min_samples_splitは2、min_samples_leafは5の時が、最もvalid_aucが高い（0.573177）ということが分かった。

### 【解説】

- 書籍本文ではrandom_stateを57で固定値としたためforループが1つで良かったのですが、今回はmin_samples_splitとmin_samples_leafを両方とも変動させたため、forループが2つになったことに注意しましょう。
- その他はほぼ書籍本編と同様の処理を行っています。
- max_depthも入れて3つのハイパーパラメータを動かした際にどの組み合わせが最も精度が高いかを検証してみても良いでしょう。

## 10.2 第8章の章末問題で自作した特徴量を使って決定木モデルを作って精度を検証してみよう。

### 【解答例】

- 第8章の章末問題で作成した中間データ（pickleファイル）を使用する

In [ ]:
# 前処理済み中間データのdictを読み取る
shomatsu_pp_data_dict = joblib.load("../intermediate/shomatsu_pp_data_dict.pkl3")

# 辞書型変数の値に格納された決定木向け中間データを読み取る
shomatsu_dtc_train_df = shomatsu_pp_data_dict["dtc"]["train"]
shomatsu_dtc_test_df = shomatsu_pp_data_dict["dtc"]["test"]

# 6:4の割合でホールドアウト法を行う.
shomatsu_dtc_train_train_df, shomatsu_dtc_train_valid_df = train_test_split(
    shomatsu_dtc_train_df,
    test_size=0.4,
    random_state=57,
    shuffle=True
)

In [ ]:
shomatsu_dtc_train_df.shape, shomatsu_dtc_train_train_df.shape, shomatsu_dtc_train_valid_df.shape, shomatsu_dtc_test_df.shape

In [ ]:
# 目的変数をtargetという変数に格納する
target = "buy_flag"

# 説明変数をfeaturesという変数に格納する
shomatsu_dtc_features = shomatsu_dtc_train_df.columns.tolist()

# customer_idとbuy_flagは説明変数ではない為削除する
shomatsu_dtc_features.remove("customer_id")
shomatsu_dtc_features.remove("buy_flag")

In [ ]:
# ハイパーパラメータチューニング
# グリッドサーチ
# 探索するハイパーパラメータの候補をリストに格納する.
shomatsu_dtc_params_list = [{"max_depth": ii, "random_state": 57} for ii in np.arange(2, 12, 1)]

# グリッドサーチの結果を格納するリスト．
shomatsu_gs_result_list = []

# グリッドサーチの実施
for params_dict in shomatsu_dtc_params_list:
    # 定義する
    tmp_shomatsu_dtc_clf = dtc(**params_dict)
    # 学習する
    tmp_shomatsu_dtc_clf.fit(shomatsu_dtc_train_train_df[shomatsu_dtc_features], shomatsu_dtc_train_train_df[target])
    
    # train-trainの予測する（predict関数）
    shomatsu_train_pred_y = tmp_shomatsu_dtc_clf.predict(shomatsu_dtc_train_train_df[shomatsu_dtc_features])
    
    # train-trainの予測する（predict_proba関数）
    shomatsu_train_proba_y = tmp_shomatsu_dtc_clf.predict_proba(shomatsu_dtc_train_train_df[shomatsu_dtc_features]).T[1]
    
    # train-validの予測する（predict関数）
    shomatsu_valid_pred_y = tmp_shomatsu_dtc_clf.predict(shomatsu_dtc_train_valid_df[shomatsu_dtc_features])
    
    # train-validの予測する（predict_proba関数）
    shomatsu_valid_proba_y = tmp_shomatsu_dtc_clf.predict_proba(shomatsu_dtc_train_valid_df[shomatsu_dtc_features]).T[1]
    
    # train-trainの検証する
    shomatsu_train_accuracy_val = accuracy_score(shomatsu_dtc_train_train_df[target], shomatsu_train_pred_y)
    shomatsu_train_auc_val = roc_auc_score(shomatsu_dtc_train_train_df[target], shomatsu_train_proba_y)
    
    # train-validの検証する
    shomatsu_valid_accuracy_val = accuracy_score(shomatsu_dtc_train_valid_df[target], shomatsu_valid_pred_y)
    shomatsu_valid_auc_val = roc_auc_score(shomatsu_dtc_train_valid_df[target], shomatsu_valid_proba_y)
    
    # リストに格納する
    shomatsu_gs_result_list += [[
        params_dict,
        params_dict["max_depth"],
        shomatsu_train_accuracy_val,
        shomatsu_train_auc_val,
        shomatsu_valid_accuracy_val,
        shomatsu_valid_auc_val
    ]]

In [ ]:
# 結果をDataFrameに格納する
shomatsu_gs_result_df = pd.DataFrame(
    shomatsu_gs_result_list,
    columns=["params_dict", "max_depth", "train_accuracy", "train_auc", "valid_accuracy", "valid_auc"]
)

# 結果を確認する
shomatsu_gs_result_df.sort_values(by="valid_auc", ascending=False).reset_index(drop=True)

In [ ]:
# Accuracyの可視化
plt.figure(figsize=(6, 5), facecolor="white", dpi=150)
plt.plot(shomatsu_gs_result_df["max_depth"], shomatsu_gs_result_df["train_accuracy"], label="train_accuracy") # train accuracy
plt.plot(shomatsu_gs_result_df["max_depth"], shomatsu_gs_result_df["valid_accuracy"], label="valid_accuracy") # valid accuracy
plt.title("Accuracy\nDTC max_depth tuning", fontsize=10)
plt.xlabel("max_depth", fontsize=10)
plt.ylabel("Accuracy", fontsize=10)
plt.legend()
plt.grid()
plt.show()

In [ ]:
# AUCの可視化
plt.figure(figsize=(6, 5), facecolor="white", dpi=150)
plt.plot(shomatsu_gs_result_df["max_depth"], shomatsu_gs_result_df["train_auc"], label="train_auc") # train auc
plt.plot(shomatsu_gs_result_df["max_depth"], shomatsu_gs_result_df["valid_auc"], label="valid_auc") # valid auc
plt.title("AUC\nDTC max_depth tuning", fontsize=10)
plt.xlabel("max_depth", fontsize=10)
plt.ylabel("AUC", fontsize=10)
plt.legend()
plt.grid()
plt.show()

In [ ]:
# train-validのAUCが最も高いハイパーパラメータの組み合わせを変数に格納する.
shomatsu_dtc_best_idx = np.argmax(shomatsu_gs_result_df["valid_auc"])
shomatsu_dtc_best_params = shomatsu_gs_result_df["params_dict"].values[shomatsu_dtc_best_idx]
shomatsu_dtc_best_score = shomatsu_gs_result_df["valid_auc"].values[shomatsu_dtc_best_idx]

# 内容を確認
print("dtc_best_params:", shomatsu_dtc_best_params)
print("dtc_best_score:", shomatsu_dtc_best_score)

# 最適化したハイパーパラメータを使用してモデルを定義する
shomatsu_dtc_clf = dtc(**shomatsu_dtc_best_params)

# 学習する
shomatsu_dtc_clf.fit(shomatsu_dtc_train_df[shomatsu_dtc_features], shomatsu_dtc_train_df[target])

# 決定木のインポータンスを確認する.
importance_df = pd.DataFrame({"feature": shomatsu_dtc_features, "importance": shomatsu_dtc_clf.feature_importances_})
importance_df = importance_df.sort_values(by="importance", ascending=False).reset_index(drop=True)
importance_df.head()

In [ ]:
# 決定木の可視化を行う
plt.figure(figsize=(20, 10), facecolor="white", dpi=150)
plot_tree(shomatsu_dtc_clf, feature_names=shomatsu_dtc_features, class_names=["not_buy", "buy"], fontsize=8, filled=True)
plt.show()

In [ ]:
# trainを予測する
shomatsu_train_pred_y = shomatsu_dtc_clf.predict(shomatsu_dtc_train_df[shomatsu_dtc_features])
shomatsu_train_proba_y = shomatsu_dtc_clf.predict_proba(shomatsu_dtc_train_df[shomatsu_dtc_features]).T[1]

# trainを検証する
shomatsu_train_accuracy_val = accuracy_score(shomatsu_dtc_train_df[target], shomatsu_train_pred_y)
shomatsu_train_auc_val = roc_auc_score(shomatsu_dtc_train_df[target], shomatsu_train_proba_y)
shomatsu_train_accuracy_val, shomatsu_train_auc_val

In [ ]:
# testを予測する
shomatsu_test_pred_y = shomatsu_dtc_clf.predict(shomatsu_dtc_test_df[shomatsu_dtc_features])
shomatsu_test_proba_y = shomatsu_dtc_clf.predict_proba(shomatsu_dtc_test_df[shomatsu_dtc_features]).T[1]

In [ ]:
# sample submitデータを読み込む
gi_sample_submit_df = pd.read_csv("../input/gi_sample_submit.csv")

In [ ]:
# submit向けDataFrameを作成し、列に予測確率を格納する
shomatsu_submit_df = shomatsu_dtc_test_df.copy()[["customer_id"]]
shomatsu_submit_df["buy_proba"] = shomatsu_test_proba_y
shomatsu_submit_df.head()

In [ ]:
gi_sample_submit_df.shape, shomatsu_submit_df.shape

In [ ]:
shomatsu_submit_df = pd.merge(
    gi_sample_submit_df.drop("buy_proba", axis=1),
    shomatsu_submit_df,
    on="customer_id",
    how="left"
).reset_index(drop=True)
shomatsu_submit_df.head()

In [ ]:
shomatsu_submit_df.shape

In [ ]:
# outputディレクトリにsubmit用ファイルを出力する
shomatsu_submit_df.to_csv(f"../output/shomatsu_submit_dtc.csv", encoding="utf-8", index=False)

### 【解説】

- 決定木モデルの構築と精度の検証については、書籍本編で紹介した方法をそのまま使うことができます。
- 他の特徴量を追加した場合も、DataFrameの形になってさえすれば、同様に流用可能です。楽できるところは楽をしてなるべく工数を減らし、その分精度向上に役立ちそうな特徴量の作成などに頭と時間を使うと良いでしょう。